In [2]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


--------------------------------------------------------------------------------

--------------------------------------------------------------------------------

#  **REVIEWS** YELP

Se separo la funcion de contar los valores duplicados para mejor legibilidad, ya que en este caso se van a estar procesando varios archivos de review

In [3]:
def nulos_vacios_yelp(df):
    """
    Este procedimiento recibe un DataFrame y muestra la cantidad de valores nulos, vacíos y la cantidad de
    valores duplicados en todas las columnas excepto en las 6, 10, 11 y 13 del DataFrame. También muestra
    el mínimo, máximo y media para columnas de tipo float e integer.

    Args:
    df (pd.DataFrame): El DataFrame a analizar.
    """
    print("Informe de Nulos, Vacíos y Duplicados:")
    print("-" * 30)

    # Total de registros en el DataFrame
    total_registros = len(df)

    # Imprimir total de registros
    print(f"Total de registros: {total_registros}")
    print("-" * 30)

    # Iterar sobre todas las columnas para mostrar nulos y vacíos
    for columna in df.columns:
        # Contar valores nulos
        nulos = df[columna].isnull().sum()

        # Contar valores vacíos (considerando que solo se cuentan si son cadenas)
        vacios = (df[columna] == '').sum() if df[columna].dtype == 'object' else 0

        # Calcular porcentajes
        porcentaje_nulos = (nulos / total_registros) * 100 if total_registros > 0 else 0
        porcentaje_vacios = (vacios / total_registros) * 100 if total_registros > 0 else 0

        # Imprimir resultados
        print(f"Columna: {columna}")
        print(f"  Valores Nulos: {nulos}")
        print(f"  Valores Vacíos: {vacios}")
        print(f"  Porcentaje Nulos: {porcentaje_nulos:.2f}%")
        print(f"  Porcentaje Vacíos: {porcentaje_vacios:.2f}%")

        # Contar y mostrar la cantidad de valores duplicados solo si no es una de las columnas con datos anidados
        # Mostrar mínimo, máximo y media si la columna es de tipo float o int
        if df[columna].dtype in ['float64', 'int64']:
            minimo = df[columna].min()
            maximo = df[columna].max()
            media = df[columna].mean()
            print(f"  Mínimo: {minimo}")
            print(f"  Máximo: {maximo}")
            print(f"  Media: {media:.2f}")
        print("-" * 30)

def duplicados_yelp_reviews(df):
    # Inicializar un diccionario para almacenar los resultados
    resultados = {}

    for columna in df.columns:
        # Revisar si la columna es una de las que queremos analizar
        if columna in [df.columns[7], df.columns[0]]:
            # Filtrar los valores no nulos antes de contar duplicados
            cantidad_duplicados = df[columna].dropna().duplicated(keep=False).sum()
            resultados[columna] = cantidad_duplicados
            print(f"Cantidad de valores duplicados en {columna} (solo no nulos): {cantidad_duplicados}")

In [4]:
def duplicados_yelp(df,x:int):
  """
  esta aplicacion va a recorrer las columnas seleccionadas para verificar si hay datos duplicados
  """
  for columna in df.columns:
    #la idea es contar si hay review_id o textos duplicados en el df(cosa que no deberia haber)
    if columna in df.columns[x]: # solo vamos a observar la columna x del DF
      # Filtrar los valores no nulos antes de contar duplicados
      cantidad_duplicados = df[columna].dropna().duplicated(keep=False).sum()
      print(f"  Cantidad de valores duplicados en {columna} (solo no nulos): {cantidad_duplicados}")
      return cantidad_duplicados

In [5]:
business = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/business.parquet')
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150343 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB


**H**abiendo definido las ciudades de antemano con el equipo, vamos a filtrar por las siguientes ciudades, ciudades cuyo nivel de comercios es el de mayor cantidad:*
- Houston
- New York
- Chicago
- Los Angeles
- Brooklyn
- San Antonio
- Dallas
- Las Vegas
- Miami
- Philadelphia

In [6]:
ciudades = ['Houston','New York','Chicago','Los Angeles','Brooklyn','San Antonio','Dallas','Las Vegas','Miami','Philadelphia']

Utilizamos esta lista para pasarlo como mascara en el df de Business y de esta manera obtener los business_id del resto de dataframes

In [7]:
business = business[business['city'].isin(ciudades)]
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14590 entries, 3 to 150336
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   14590 non-null  object 
 1   name          14590 non-null  object 
 2   address       14590 non-null  object 
 3   city          14590 non-null  object 
 4   state         14590 non-null  object 
 5   postal_code   14590 non-null  object 
 6   latitude      14590 non-null  float64
 7   longitude     14590 non-null  float64
 8   stars         14590 non-null  float64
 9   review_count  14590 non-null  int64  
 10  is_open       14590 non-null  int64  
 11  attributes    13417 non-null  object 
 12  categories    14581 non-null  object 
 13  hours         11801 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 1.7+ MB


Nos aseguramos que no hayan id de negocios duplicados

In [31]:
duplicados_yelp(business,0)

  Cantidad de valores duplicados en business_id (solo no nulos): 0


0

Guardamos el df limpio en un parquet

In [32]:
business.to_parquet('/content/drive/MyDrive/Proyecto_final/data_clean/business_clean.parquet')

sabiendo que no hay datos duplicados, resta pasar los ID de los negocios, a una lista, para luego filtrarlos en los archivos de los review, user, etc

In [9]:
lista_id_comercios = business['business_id'].tolist()

# se realiza la revision de los archivos de review0,review1, [...], review6

In [66]:
df_review0 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review0.parquet')
df_review0 = df_review0[df_review0['business_id'].isin(lista_id_comercios)]

df_review1 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review1.parquet')
df_review1 = df_review1[df_review1['business_id'].isin(lista_id_comercios)]

df_review2 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review2.parquet')
df_review2 = df_review2[df_review2['business_id'].isin(lista_id_comercios)]

df_review3 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review3.parquet')
df_review3 = df_review3[df_review3['business_id'].isin(lista_id_comercios)]

df_review4 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review4.parquet')
df_review4 = df_review4[df_review4['business_id'].isin(lista_id_comercios)]

df_review5 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review5.parquet')
df_review5 = df_review5[df_review5['business_id'].isin(lista_id_comercios)]

df_review6 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/review6.parquet')
df_review6 = df_review6[df_review6['business_id'].isin(lista_id_comercios)]

In [67]:
reviews_general = pd.concat([df_review0,df_review1,df_review2,df_review3,df_review4,df_review5,df_review6], ignore_index=True)
reviews_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968159 entries, 0 to 968158
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    968159 non-null  object        
 1   user_id      968159 non-null  object        
 2   business_id  968159 non-null  object        
 3   stars        968159 non-null  int64         
 4   useful       968159 non-null  int64         
 5   funny        968159 non-null  int64         
 6   cool         968159 non-null  int64         
 7   text         968159 non-null  object        
 8   date         968159 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 66.5+ MB


In [7]:
nulos_vacios_yelp(reviews_general)

Informe de Nulos, Vacíos y Duplicados:
------------------------------
Total de registros: 968159
------------------------------
Columna: review_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: user_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: business_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: stars
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
  Mínimo: 1
  Máximo: 5
  Media: 3.79
------------------------------
Columna: useful
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
  Mínimo: 0
  Máximo: 320
  Media: 1.30
------------------------------
Columna: funny
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaj

**Como se puede ver en la data, no hay valores nulos o faltantes en cada uno de los archivos revisados de reviews, todas las columnas dan 0 valores faltantes o strings vacios**

Lo que se puede revisar ahora es:
- si hay datos duplicados, mas precisamente si hay review_id duplicados o textos duplicados, ya que seria raro encontrar dos comentarios identicos de dos personas distintas

In [10]:
duplicados_yelp_reviews(reviews_general)

Cantidad de valores duplicados en review_id (solo no nulos): 0
Cantidad de valores duplicados en text (solo no nulos): 3450


chequeamos si los duplicados no son mensajes cortos, como 'good service', 'i don't like', etc. o algun otro mensaje generico

In [17]:
reviews_general['text'].value_counts()

text
Omg!There's food was good!Ryan is awesome aways good service also he is so cute! We will come back again for him! I love you!!!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

##  

---
##  
 - *Dado que son mensajes elaborados lo que vamos a hacer es realizar una comparacion de estos datos para verificar que la fila sea identica*

In [ ]:
indices_duplicados = reviews_general[reviews_general['text'].duplicated(keep=False)].index.tolist()

##  
solo para chequear que hayan valores....

In [19]:
indices_duplicados[0:5]

[589, 1983, 2423, 2567, 2651]

se revisa algunas filas para corroborar si los datos de las otras columnas coinciden en su mayoria

In [21]:
duplicados = reviews_general.iloc[indices_duplicados]

Vamos a elegir algunos datos al azar

In [22]:
duplicados

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
589,WeTHLEdLdP3Y3AXWWCsx4g,UizCd11Qoo74BKARrZp7HA,vUrTGX_7HxqeoQ_6QCVz6g,5,0,0,1,We are visiting in Philadelphia from Californi...,2018-05-28 21:27:38
1983,NY8UWeooFGlwpFP0_auYxQ,xA1eoU2N9Z8XbkX81sPA2A,cRJIf2i9LX8X8DKDG2qjiQ,1,1,0,0,I'm very displeased with my experience here th...,2017-03-26 16:41:08
2423,Iozmt2sY2Khfupiqpj1Rqg,4UAqdgiCe1jwI1IQl96MIA,aAMop7hpw7rtLL0bnS5lLQ,1,1,1,0,I took a cooking class at La Cucina and it was...,2015-09-24 23:36:04
2567,8elcvgZ_HoXoKUbzTla1Gw,vCVohQjM84c2XuUzCdPWlQ,OHzX-ZD9qyoeoxR8Z0dlIA,4,0,0,0,Khyber was a great find for a trip to Philly. ...,2017-08-09 00:02:22
2651,g_riW0ySQ3mZ_NpwOIRrvg,y8Fhw_wEYfLqid7945vjfQ,jL_NufxqXi-BpW5uXKsPwQ,3,0,0,0,The previous review said Rite Aide is bad. I k...,2016-01-17 21:51:29
...,...,...,...,...,...,...,...,...,...
966782,NKPSke1_9HbIGnTLd_-K4g,xRsK05J7MEo7lV1kru3bLQ,ziyXteCmNmRprs10buP2iQ,5,1,0,1,"5 stars for food, 4 stars for service. We were...",2021-11-25 03:14:01
966808,61rf0WGcQwhfMA849xhOQQ,x5KPwR_jCVFodbos1bq9TA,ytynqOUb3hjKeJfRj5Tshw,4,0,0,0,This place is crowded as hell. I got some expe...,2019-07-07 15:48:22
967273,0K4jUPI4SsrQ0PfBLng30Q,9y1U8xphysbnssT8Vo-QOg,ZGH8MzJa7Q7iIpmtzBAJAQ,1,1,2,0,I came to MB and I ordered food that totaled $...,2017-09-30 01:19:20
967559,9axZ3zl103SLs5Fi9PA2aw,SIGMyncbCt2P7Sq3sCnrvw,w9_EQB0SuAFYxPMcfY_P_g,3,1,0,0,Mugshots follows the age old saying...life is ...,2016-05-05 20:14:59


- **elijo las dos siguientes para chequear sus filas duplicadas:** *I'm very displeased with my experience here / This place is crowded as hell. I got some*


In [25]:
duplicado = reviews_general[reviews_general['text'].str.contains("I'm very displeased with my experience here")]
duplicado

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
1983,NY8UWeooFGlwpFP0_auYxQ,xA1eoU2N9Z8XbkX81sPA2A,cRJIf2i9LX8X8DKDG2qjiQ,1,1,0,0,I'm very displeased with my experience here th...,2017-03-26 16:41:08
3420,6zWjE7aDAOUz94d3vwTRzg,xA1eoU2N9Z8XbkX81sPA2A,cRJIf2i9LX8X8DKDG2qjiQ,1,4,0,0,I'm very displeased with my experience here th...,2017-03-27 17:37:23


In [26]:
duplicado = reviews_general[reviews_general['text'].str.contains("This place is crowded as hell. I got some")]
duplicado

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
963477,CUIuu3Fy4uPZ0itUwG6rkg,x5KPwR_jCVFodbos1bq9TA,ytynqOUb3hjKeJfRj5Tshw,4,1,0,0,This place is crowded as hell. I got some expe...,2019-09-08 13:39:59
966808,61rf0WGcQwhfMA849xhOQQ,x5KPwR_jCVFodbos1bq9TA,ytynqOUb3hjKeJfRj5Tshw,4,0,0,0,This place is crowded as hell. I got some expe...,2019-07-07 15:48:22


**Claramente los datos son casi identicos, usualmente lo unico que suele cambiar es la fecha y el review_id, por lo que podriamos decir que son datos que se ingresaron mal por algun bug de la aplicacion.**

In [27]:
reviews_sin_duplicados = reviews_general.drop_duplicates(subset='text', keep=False)

In [29]:
reviews_sin_duplicados.to_parquet('/content/drive/MyDrive/Proyecto_final/data_clean/reviews_clean.parquet')

**N**os queda obtener los usuarios que interactuaron para filtrarlos en los archivos de USER

In [ ]:
usuarios_sin_duplicados = reviews_sin_duplicados.drop_duplicates(subset='user_id', keep=False)
usuarios_sin_duplicados = usuarios_sin_duplicados['user_id'].tolist()

#  

#  

#  **USERS** YELP

filtramos los usuarios que obtuvimos del df anterior

In [6]:
df_user0 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/user0.parquet')
df_user0 = df_user0[df_user0['user_id'].isin(usuarios_sin_duplicados)]

df_user1 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/user1.parquet')
df_user1 = df_user1[df_user1['user_id'].isin(usuarios_sin_duplicados)]

df_user2 = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/user2.parquet')
df_user2 = df_user2[df_user2['user_id'].isin(usuarios_sin_duplicados)]

In [7]:
df_user_general = pd.concat([df_user0,df_user1,df_user2], ignore_index=True)
df_user_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174034 entries, 0 to 174033
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             174034 non-null  string 
 1   name                174034 non-null  string 
 2   review_count        174034 non-null  int64  
 3   yelping_since       174034 non-null  string 
 4   useful              174034 non-null  int64  
 5   funny               174034 non-null  int64  
 6   cool                174034 non-null  int64  
 7   elite               174034 non-null  string 
 8   friends             174034 non-null  string 
 9   fans                174034 non-null  int64  
 10  average_stars       174034 non-null  float64
 11  compliment_hot      174034 non-null  int64  
 12  compliment_more     174034 non-null  int64  
 13  compliment_profile  174034 non-null  int64  
 14  compliment_cute     174034 non-null  int64  
 15  compliment_list     174034 non-nul

**En este caso solo vamos a revisar la columna user_id, que no deberia tener valores duplicados**

In [11]:
duplicados_yelp(df_user_general,0)

  Cantidad de valores duplicados en user_id (solo no nulos): 10298


10298

In [12]:
duplicados = df_user_general[df_user_general['user_id'].duplicated(keep=False)]
duplicados

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,65uUyG9yuF0rdMh0z7ajaA,Don,715,2005-11-30 20:54:16,808,258,712,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2015","cvzKVWaGFysF1iZ1RUvsBg, p5TRjvekkxtrh2p3eg9DFA...",37,...,9,4,4,4,24,48,115,115,19,15
1,aF3mEXDJuILmeF-9PcxAsQ,C.J.,250,2007-06-26 15:50:11,507,155,150,2010,"fXz9gKz84zBph3bGITTSgg, YIlY03dS0YrJtba8N-tMGw...",15,...,5,1,0,1,6,13,10,10,5,0
2,XLs_PhrJ7Qwn_RfgMM7Djw,Weili,90,2009-07-12 14:34:54,71,21,19,,"gTX68_Us2eI2KjY21aOQ7g, e4nHt3ErihX7II-MHXVM1w...",1,...,0,0,0,0,1,3,0,0,0,0
3,sgCZmOkdHBiaKaE6ZKVBaA,Chris,100,2005-12-12 06:30:39,100,19,25,,"AbOmMCW6mEMc6pY3o4HFNQ, HK2JAemtt2mUinNNQVjdPw...",2,...,1,0,0,0,2,0,3,3,1,0
4,2Yw9-N2VYZFqg2ylhpb8NA,Kathryn,11,2011-01-12 23:13:34,34,3,3,,"XX2vDV2XT1cNXxyrTE2qGw, bg2GfRJ99UuPs4p92TwO1g...",1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174029,QLW8Qiz0plIlaM199-CA_g,Hanna,63,2013-08-15 22:51:53,54,11,18,,"I8vLntrav41PcddTq-v5pg, 0fcm6BsexkXLCwklymmsEQ...",1,...,0,0,0,0,0,1,1,1,0,0
174030,4VpUsb9CjWV2kUWokGsV1g,Kue,7,2013-08-24 12:58:10,1,0,1,,"mCF3CDKJ0-z7w8Y5GHl6nQ, 4rBdMBflCZzzBLLhhlFOgA...",0,...,0,0,0,0,0,0,0,0,0,0
174031,x0b2pgaOSsClQif5PuYnnw,Shaterra,1,2016-11-23 15:21:18,1,0,0,,"SEb7JbP70yOCOtWEKPBwxA, AUG3xNAkNnuUTogx5vMa3g...",0,...,0,0,0,0,0,0,0,0,0,0
174032,7N8RIWNAXvPHrTRJi7LphQ,Jason,53,2015-02-22 14:11:54,16,6,3,,"ZZN-DVejzp8FTOk0_A8KWg, mvHstmfVgUcuV7U9nqw5Bg...",0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
duplicados['user_id'].value_counts()

,count
user_id,
tNIhbQMww3GbWZipCB9_pg,5
nYgzvpBQZqaWDWz3EIhpow,5
LbcW52gH3_xP0x6CkWEl4Q,5
G8UL6NO3kqnlhweKvoV3Dw,5
9oo8nJ59_w3T-jLU_kKYVw,5
...,...
n8hYN-uEbHKYYfR_XHlRzg,2
oSC_CCDQlImK87W38JiPtg,2
4VpUsb9CjWV2kUWokGsV1g,2


In [15]:
duplicado = df_user_general[df_user_general['user_id'].str.contains("tNIhbQMww3GbWZipCB9_pg")]
duplicado

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
1922,tNIhbQMww3GbWZipCB9_pg,Priya,5,2009-09-22 00:09:46,0,0,0,,"2QQb8k9eR7kM0fiYGsaYuw, t9LHFBkjCUD1dk40kb07hA...",0,...,0,0,0,0,0,0,0,0,0,0
168598,tNIhbQMww3GbWZipCB9_pg,Priya,5,2009-09-22 00:09:46,0,0,0,,"2QQb8k9eR7kM0fiYGsaYuw, t9LHFBkjCUD1dk40kb07hA...",0,...,0,0,0,0,0,0,0,0,0,0
169948,tNIhbQMww3GbWZipCB9_pg,Priya,5,2009-09-22 00:09:46,0,0,0,,"2QQb8k9eR7kM0fiYGsaYuw, t9LHFBkjCUD1dk40kb07hA...",0,...,0,0,0,0,0,0,0,0,0,0
171298,tNIhbQMww3GbWZipCB9_pg,Priya,5,2009-09-22 00:09:46,0,0,0,,"2QQb8k9eR7kM0fiYGsaYuw, t9LHFBkjCUD1dk40kb07hA...",0,...,0,0,0,0,0,0,0,0,0,0
172648,tNIhbQMww3GbWZipCB9_pg,Priya,5,2009-09-22 00:09:46,0,0,0,,"2QQb8k9eR7kM0fiYGsaYuw, t9LHFBkjCUD1dk40kb07hA...",0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
duplicado = df_user_general[df_user_general['user_id'].str.contains("Kst_srPw7GdYydMFYdCtzw")]
duplicado

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
5671,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0
174033,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
users_sin_duplicados = df_user_general.drop_duplicates(subset='user_id', keep=False)
users_sin_duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163736 entries, 706 to 166506
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             163736 non-null  string 
 1   name                163736 non-null  string 
 2   review_count        163736 non-null  int64  
 3   yelping_since       163736 non-null  string 
 4   useful              163736 non-null  int64  
 5   funny               163736 non-null  int64  
 6   cool                163736 non-null  int64  
 7   elite               163736 non-null  string 
 8   friends             163736 non-null  string 
 9   fans                163736 non-null  int64  
 10  average_stars       163736 non-null  float64
 11  compliment_hot      163736 non-null  int64  
 12  compliment_more     163736 non-null  int64  
 13  compliment_profile  163736 non-null  int64  
 14  compliment_cute     163736 non-null  int64  
 15  compliment_list     163736 non-null  

In [24]:
nulos_vacios_yelp(users_sin_duplicados)

Informe de Nulos, Vacíos y Duplicados:
------------------------------
Total de registros: 163736
------------------------------
Columna: user_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: name
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: review_count
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
  Mínimo: 0
  Máximo: 12868
  Media: 20.59
------------------------------
Columna: yelping_since
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: useful
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
  Mínimo: 0
  Máximo: 44130
  Media: 34.68
------------------------------
Columna: funny
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%

In [25]:
users_sin_duplicados.to_parquet('/content/drive/MyDrive/Proyecto_final/data_clean/users_clean.parquet')

**H**acemos lo mismo con checkin

In [11]:
checkin = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/checkin.parquet')
checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [12]:
df_checkin = checkin[checkin['business_id'].isin(lista_id_comercios)]
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12911 entries, 14 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  12911 non-null  object
 1   date         12911 non-null  object
dtypes: object(2)
memory usage: 302.6+ KB


In [15]:
nulos_vacios_yelp(df_checkin)

Informe de Nulos, Vacíos y Duplicados:
------------------------------
Total de registros: 12911
------------------------------
Columna: business_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: date
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------


In [16]:
duplicados_yelp(df_checkin,0)

  Cantidad de valores duplicados en business_id (solo no nulos): 0


0

In [30]:
df_checkin.to_parquet('/content/drive/MyDrive/Proyecto_final/data_clean/checkin_clean.parquet')

**P**or ultimo, filtramos tip con la lista de los id de los comercios

In [18]:
tip = pd.read_parquet('/content/drive/MyDrive/Proyecto_final/data/tip.parquet')
tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [19]:
df_tip = tip[tip['business_id'].isin(lista_id_comercios)]
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118644 entries, 2 to 908895
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           118644 non-null  object        
 1   business_id       118644 non-null  object        
 2   text              118644 non-null  object        
 3   date              118644 non-null  datetime64[ns]
 4   compliment_count  118644 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 5.4+ MB


In [20]:
nulos_vacios_yelp(df_tip)

Informe de Nulos, Vacíos y Duplicados:
------------------------------
Total de registros: 118644
------------------------------
Columna: user_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: business_id
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: text
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: date
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
------------------------------
Columna: compliment_count
  Valores Nulos: 0
  Valores Vacíos: 0
  Porcentaje Nulos: 0.00%
  Porcentaje Vacíos: 0.00%
  Mínimo: 0
  Máximo: 3
  Media: 0.01
------------------------------


In [21]:
duplicados_yelp(df_tip,2)

  Cantidad de valores duplicados en text (solo no nulos): 6865


6865

chequeamos los indices y verificamos que la longitud de la lista coincida con la cantidad de datos duplicados que nos da la funcion

In [22]:
indices_duplicados = df_tip[df_tip['text'].duplicated(keep=False)].index.tolist() # guardamos los indices duplicados
lista_duplicado = [] #almacenamos aca los indices que nos va a devolver la funcion duplicated().index.tolist()
lista_duplicado.append(indices_duplicados) # los agregamos a la lista
len(lista_duplicado[0])

6865

Ahora si usamos la mascara para filtrar los datos duplicados del df

In [23]:
duplicados = tip[tip.index.isin(lista_duplicado[0])]
duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6865 entries, 113 to 908888
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           6865 non-null   object        
 1   business_id       6865 non-null   object        
 2   text              6865 non-null   object        
 3   date              6865 non-null   datetime64[ns]
 4   compliment_count  6865 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 321.8+ KB


#  

Vamos a elegir nuevamente algunos comentario al azar para corroborar si coinciden el resto de caracteristicas

In [24]:
duplicados

,user_id,business_id,text,date,compliment_count
113,5Si-3Rt03p8r7fGYd-1EVw,htO_nlxkEsYHzDrtbiUxew,Great coffee,2013-02-05 16:52:57,0
160,KqzgupvHOVfYQhefLBQxzg,yPOVNQu0iU6BR8RPIfIE3Q,Yum yum yum!,2012-08-01 13:42:12,0
285,0jG8jIMK_61QovBdCiA_2Q,9A5Gw0At6so0x-vWM0_JZw,My favorite!!!,2017-03-26 14:53:47,0
659,3mohph-KcszKKYZNaNrgUg,Og1HekJMHKqsvtWaUp9V8g,Yummy,2016-02-27 14:38:46,0
660,zZxXoKr5TzsAYK5e5Z7hyA,EzjysPg2-lVX1E0ibStUXw,Never disappoints!,2015-10-08 21:55:29,0
...,...,...,...,...,...
908217,HTJ6NVxFGYyVhEgMH8G3lg,iksVwRfpWymIUUFqw0tXpw,Love the oxtail hotpot.......,2019-04-16 19:58:35,0
908255,4AbzWRHJ_F_8JmnCnd2hDQ,K2svtoFXk68mFIIwLjG36g,...,2018-02-12 12:27:10,0
908391,A_TwClVTDfQG5lH_jQzU0Q,qQO7ErS_RAN4Vs1uX0L55Q,Cash only!,2017-09-27 09:01:10,0
908647,SRYQb6GDrUgP5OoHSFcdRQ,qY-BUQY-SFBaSrFHowF3nA,Delicious,2018-04-23 19:53:44,0


In [25]:
datos_random = tip[tip['text']=="Tacos"]
datos_random

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0
13341,kVqjvUueC7hWZ1_IjkGjvg,mFIffumYpySeZnQ3ckqpDA,Tacos,2010-02-20 16:54:36,0
284890,cuAHifRAkYY9iEVZkRlpXA,vhDWGF-8BfsxvS7Zo5Wv2w,Tacos,2018-07-29 04:40:56,0
373167,Rr4cLb6Go91FT134o6RsKg,qFwKNXKt2fvCatrLAqDONg,Tacos,2012-05-13 01:47:47,0
412667,HXB_ByCCp8cZm1WMjKn1Tw,SCjUeZfjafJ88fWWQjRh2w,Tacos,2011-08-26 21:35:35,0
425498,M5HoGdy-mNOV7EHhxB9P9A,HwyR8zWjIomCDrHAJ0cXSQ,Tacos,2013-05-24 03:24:25,0
833946,ccMnj_hT1QJfUuMrL9644g,HF9LX5776QYYK70tyIL8gw,Tacos,2013-01-28 22:03:37,0
906357,R53MDGKiur16lY7g_B74Eg,6gCQvovXf3wmrfwuTyeLlQ,Tacos,2012-02-16 18:45:07,0


In [26]:
datos_random1 = tip[tip['text']=="Great food and service."]
datos_random1

,user_id,business_id,text,date,compliment_count
24773,G52P5bsFM3AjN64MKpPS0Q,2fuQnVPwhU_zzfUog_JGsw,Great food and service.,2016-08-28 21:41:57,0
52795,xWR-GSPPlt7taGuM1vk_0g,8kUh6TROemLfbVR_ewVVLg,Great food and service.,2015-10-24 00:47:48,0
67037,wemLDAZ-sPdxlSt9VvkeNg,5oX4G1cptixPZecih9L3dg,Great food and service.,2017-01-28 19:37:07,0
89924,xKVTd1u9A4ucHp__g4U-Ew,A5sXEcVHYnmNDs0bwb7u-w,Great food and service.,2018-01-27 11:59:31,0
173407,SiXy8yxpjhZPcTeI0BjPeQ,s63cVEM7S0SSBQIkUIsfJQ,Great food and service.,2014-04-03 13:32:26,0
...,...,...,...,...,...
893407,ezuHFcqzu9PFOq2ajq1Geg,PzhPMkaNYiKDTHoTG0r8rw,Great food and service.,2016-04-30 23:22:42,0
904078,rhAMh47RIlUIg0lyyGAXQQ,qt_E6txwQ1h62wyv8701UQ,Great food and service.,2019-04-22 01:10:57,0
905776,ejo3D0lanSoG0PWDmHm03g,scoHdh0XDA_W7xAZW3zAKQ,Great food and service.,2021-06-26 20:22:11,0
908754,EDicnxs2NPxktUc4KgHDdg,4A_1kNLFVYmQ-zfz93huBg,Great food and service.,2016-06-26 15:42:14,0


dado que son como sugerencias de diferentes lugares y diferentes usuarios, podemos dejarlo sin efecto
Lo que restaria revisar es si la caracteristica de conteo de cumplidos tiene mas datos que 0

In [27]:
tip.compliment_count.value_counts()

,count
compliment_count,
0,898376
1,9821
2,617
3,82
4,14
5,3
6,2


In [29]:
df_tip.to_parquet('/content/drive/MyDrive/Proyecto_final/data_clean/tip_clean.parquet')